In [21]:
import pandas as pd
import re

In [3]:
jeremyDataPath='/accounts/ekarakoy/DATA/NOMAD/JeremyOCx_DF.pkl'
dfJeremy=pd.read_pickle(jeremyDataPath)

In [20]:
cols = dfJeremy.columns.tolist()

In [38]:
dfJeremy.columns

Index(['cruise', 'lat', 'lon', 'type', 'chl', 'rrs443', 'rrs489', 'rrs510',
       'rrs555', 'rrs670', 'rrs469', 'rrs486', 'rrs516', 'rrs520', 'rrs547',
       'rrs550', 'rrs560', 'rrs565', 'rrs411', 'Sdg', 'adg405', 'adg411',
       'adg443', 'adg489', 'adg510', 'adg530', 'adg555', 'eta', 'bbp405',
       'bbp411', 'bbp443', 'bbp489', 'bbp510', 'bbp530', 'bbp555', 'aph405',
       'aph411', 'aph443', 'aph489', 'aph510', 'aph530', 'aph555', 'aph670',
       'aph683', 'zeu', 'kpar', 'kd405', 'kd411', 'kd443', 'kd489', 'kd510',
       'kd530', 'kd555', 'Datetime', 'mxBl2GrSwf'],
      dtype='object')

In [105]:
def GetBandsRe(startswith='rrs'):
    return re.compile('^%s([0-9]+)' % startswith)

def GetBands(reFinder, sort=True):
    bandlist = [reFinder.findall(c)[0] for c in cols if reFinder.findall(c)]
    if sort:
        bandlist.sort()
    return bandlist

def GetSingleBandDataFrame(df, band):
    return df.filter(regex='([\w]+%d)' % band)

In [39]:
rrsbands = GetBands(GetBandsRe())
adgbands = GetBands(GetBandsRe('adg'))
aphbands = GetBands(GetBandsRe('aph'))
bbpbands = GetBands(GetBandsRe('bbp'))

In [48]:
# finding common bands
commonbands = list(set(rrsbands).intersection(adgbands).intersection(aphbands).intersection(bbpbands))
commonbands.sort()
commonbands

['411', '443', '489', '510', '555']

In [50]:
# FROM IDL routines get_aw.pro and get_bw.pro
aw = {405: 0.00545821, 411: 0.00471065, 443: 0.00709810, 489: 0.0149089,
      510: 0.0325987, 530: 0.0435704, 555: 0.0596488, 670: 0.440188, 
      683: 0.479974
     }
bw = {405: 0.00360145, 411: 0.00337976, 443: 0.00244466, 489: 0.00159540,
      510: 0.00133039, 530: 0.00112671, 555: 0.000923288, 670: 0.000409298,
      683: 0.000376691
     }

In [57]:
for band in commonbands:
    dfJeremy['a%s' % band] = dfJeremy['aph%s' % band] +\
                             dfJeremy['adg%s' % band] + \
                             aw[int(band)]
    dfJeremy['bb%s' % band] = dfJeremy['bbp%s' % band] + bw[int(band)]

In [86]:
filterString = '^(([a-b][\w]*|rrs)(' + '|'.join(commonbands) +')|chl|type)'
filterString

'^(([a-b][\\w]*|rrs)(411|443|489|510|555)|chl|type)'

In [87]:
dfJeremy.filter(regex=filterString).columns

Index(['type', 'chl', 'rrs443', 'rrs489', 'rrs510', 'rrs555', 'rrs411',
       'adg411', 'adg443', 'adg489', 'adg510', 'adg555', 'bbp411', 'bbp443',
       'bbp489', 'bbp510', 'bbp555', 'aph411', 'aph443', 'aph489', 'aph510',
       'aph555', 'a411', 'bb411', 'a443', 'bb443', 'a489', 'bb489', 'a510',
       'bb510', 'a555', 'bb555'],
      dtype='object')

In [92]:
dfGIOP = dfJeremy.filter(regex=filterString)

In [93]:
for b in commonbands:
    rrs = dfGIOP['rrs%s' %b]
    dfGIOP['rrs%ssub' %b] = rrs / (0.52 + 1.7 *rrs)

In [124]:
dfGIOP.to_pickle('./pickleJar/dfGIOP.pkl')
dfJeremy.to_pickle('./pickleJar/dfJeremy.pkl')